In [228]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer
nltk.download('punkt')
nltk.download('punkt_tab')
from tqdm import tqdm
tqdm.pandas()



# data = pd.read_csv("C:\\Users\\1\\Desktop\\proga\\ЛШ\\проект ЛШ\\LizaAlert.csv")
data = pd.read_csv("C:\\Users\\1\\Desktop\\proga\\ЛШ\\проект ЛШ\\updated_file.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [229]:
data

,Unnamed: 0,id_hash,gender,age,location,status,date_search,date_of_loss,last_search_date,search_period,head,content,date_of_find
0,0,002a7a0fd2047c4812ee06974cfec87266c73307ac4d29...,жен,65.0,NaN,пропал(а),"19 сен 2023, 00:42",NaN,"19 сен 2023, 00:42",0 days 00:00:00,NaN,NaN,NaN
1,1,008c48640c96927ecb37bf877179e2e506052ea235e4d7...,муж,38.0,пгт. Кавалерово,пропал(а),"12 июн 2024, 06:25",9 июня 2024,"12 июн 2024, 06:26",0 days 00:01:00,NaN,NaN,NaN
2,2,00fdf06a413b627f8a53c77ce9dcde274b94aa895d6f15...,муж,NaN,NaN,NaN,"15 авг 2018, 02:00",NaN,"17 авг 2018, 08:44",2 days 06:44:00,NaN,NaN,NaN
3,3,012742c048aa46328fbca7f18376b6db990499a31ca158...,муж,69.0,NaN,пропал(а),"15 сен 2023, 06:57",24 июня 2023,"30 июн 2023, 08:29",-77 days +01:32:00,NaN,NaN,NaN
4,4,0228b07e42be793dc3259b75fe30468085922a0959c24f...,жен,45.0,NaN,жив(а),"02 июл 2021, 11:53",30 июня 2021,"05 июл 2021, 06:31",2 days 18:38:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40012,198,f9ae1b4760f033ebb66106b97da481f7bd04016714a24c...,муж,38.0,NaN,NaN,"23 июн 2017, 19:02",24 мая 2017,"23 июн 2017, 19:02",0 days 00:00:00,NaN,NaN,NaN
40013,199,faf941ba5fed320799aa39d7d135e17297d45101f08c69...,муж,78.0,г. Нижний Тагил,NaN,"20 апр 2016, 13:18",18 апреля 2016,"22 апр 2016, 16:07",2 days 02:49:00,NaN,NaN,NaN
40014,200,fb3ef26817b26b1c91e286780e474b7c4312d6af9df4da...,муж,45.0,NaN,NaN,"17 мар 2021, 17:40",11 марта 2021,"17 мар 2021, 17:40",0 days 00:00:00,NaN,NaN,NaN
40015,201,fc1df6c8cab366b4b986f373d678ed390d122c5fa5c1c2...,муж,43.0,г. Сухой Лог,NaN,"06 янв 2023, 12:55",1 января 2023,"25 июл 2023, 15:33",200 days 02:38:00,NaN,NaN,NaN


In [ ]:
morph = MorphAnalyzer()


def determine_name_part_gender(name_part, head_line):
        """ Определение пола по части имени """
        name_part_check = 'неопр'
        number_people = 0
        flag = False
        k = 0
        # определяем пол по части имени
        for i in head_line:
                if name_part in str(morph.parse(i)[0].tag):
                        name_part_check = 'муж' if 'masc' in str(morph.parse(i)[0].tag) else 'жен'
                else:
                        if flag == True and k >= 2:
                                flag2 = False
                                number_people += 1
                        k += 1

        # если больше двух имен в строке, то пропало несколько человек
        if number_people >= 2:
                name_part_check = 'мн'

        return name_part_check


def determine_first_word_gender(head_line):
        """ Определение пола по первому слову """
        first_check = 'неопр'
        first_word = str(morph.parse(head_line[0])[0].tag)
        # если первое слово имеет род, то возвращаем его
        if ('ADJS' in first_word) or ('VERB' not in first_word) or ('PRTS' not in first_word):
                if 'plur' in first_word:
                        first_check = 'мн'
                elif 'femn' in first_word:
                        first_check = 'жен'
                elif 'masc' in first_word:
                        first_check = 'муж'

        return first_check            
        

def determine_gender(r_gender, r_head):
        """ Функция для приведения пола пропавших людей к одной из
            категорий муж, жен, мн, неопр по содержимому столбца head
        """
        first_check = 'неопр'
        name_check = 'неопр'
        patronim_check = 'неопр'
        surname_check = 'неопр'
        
        # обрабатываем только те столбцы, где есть значения в head
        if not pd.isna(r_head):
                try:
                        head_line = [w.lower() for w in word_tokenize(r_head, language='russian') if ((w.isalpha() or w.isdigit()) and w != 'Re')]
                        # прогоняем функцию для имени, фамилии отчества
                        first_check = determine_first_word_gender(head_line)
                        name_check = determine_name_part_gender('Name', head_line)
                        surname_check = determine_name_part_gender('Surn', head_line)
                        patronim_check = determine_name_part_gender('Patr', head_line)
                
                except Exception as e:
                        print(e)

                variants = [first_check, name_check, patronim_check, surname_check]

                # выбираем первый из вариантов, где пол определен
                for i in variants:
                        if i != 'неопр':
                                return i

                return variants[-1]

In [232]:
data['gender_new'] = data.progress_apply(lambda row: determine_gender(row['gender'], row['head']), axis=1)

 12%|█▏        | 4797/40017 [00:16<01:54, 308.20it/s]

list index out of range


100%|██████████| 40017/40017 [02:04<00:00, 321.92it/s]  


In [233]:
data['gender'].value_counts()

gender
male      31200
муж        5113
жен        2267
female     1437
Name: count, dtype: int64

In [234]:
data['gender_new'].value_counts()

gender_new
муж      20827
жен      10939
мн         774
неопр       94
Name: count, dtype: int64

In [235]:
data['gender'] = data.apply(lambda x: x['gender_new'] if x['gender'] in ['male', 'female'] else x['gender'], axis=1)
data['gender'].value_counts()

gender
муж      25940
жен      13206
мн         774
неопр       94
Name: count, dtype: int64

In [239]:
data = data.drop(columns=['gender_new'])
data

,Unnamed: 0,id_hash,gender,age,location,status,date_search,date_of_loss,last_search_date,search_period,head,content,date_of_find
0,0,002a7a0fd2047c4812ee06974cfec87266c73307ac4d29...,жен,65.0,NaN,пропал(а),"19 сен 2023, 00:42",NaN,"19 сен 2023, 00:42",0 days 00:00:00,NaN,NaN,NaN
1,1,008c48640c96927ecb37bf877179e2e506052ea235e4d7...,муж,38.0,пгт. Кавалерово,пропал(а),"12 июн 2024, 06:25",9 июня 2024,"12 июн 2024, 06:26",0 days 00:01:00,NaN,NaN,NaN
2,2,00fdf06a413b627f8a53c77ce9dcde274b94aa895d6f15...,муж,NaN,NaN,NaN,"15 авг 2018, 02:00",NaN,"17 авг 2018, 08:44",2 days 06:44:00,NaN,NaN,NaN
3,3,012742c048aa46328fbca7f18376b6db990499a31ca158...,муж,69.0,NaN,пропал(а),"15 сен 2023, 06:57",24 июня 2023,"30 июн 2023, 08:29",-77 days +01:32:00,NaN,NaN,NaN
4,4,0228b07e42be793dc3259b75fe30468085922a0959c24f...,жен,45.0,NaN,жив(а),"02 июл 2021, 11:53",30 июня 2021,"05 июл 2021, 06:31",2 days 18:38:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40012,198,f9ae1b4760f033ebb66106b97da481f7bd04016714a24c...,муж,38.0,NaN,NaN,"23 июн 2017, 19:02",24 мая 2017,"23 июн 2017, 19:02",0 days 00:00:00,NaN,NaN,NaN
40013,199,faf941ba5fed320799aa39d7d135e17297d45101f08c69...,муж,78.0,г. Нижний Тагил,NaN,"20 апр 2016, 13:18",18 апреля 2016,"22 апр 2016, 16:07",2 days 02:49:00,NaN,NaN,NaN
40014,200,fb3ef26817b26b1c91e286780e474b7c4312d6af9df4da...,муж,45.0,NaN,NaN,"17 мар 2021, 17:40",11 марта 2021,"17 мар 2021, 17:40",0 days 00:00:00,NaN,NaN,NaN
40015,201,fc1df6c8cab366b4b986f373d678ed390d122c5fa5c1c2...,муж,43.0,г. Сухой Лог,NaN,"06 янв 2023, 12:55",1 января 2023,"25 июл 2023, 15:33",200 days 02:38:00,NaN,NaN,NaN


In [240]:
data.to_csv('updated_file_1.csv')

# МУСОР

In [226]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
for i in 'Лоскутникова Надежда Владимировна'.split():
    print(str(morph.parse(i)[0].tag))


NOUN,anim,masc,Sgtm,Surn sing,gent
NOUN,inan,femn sing,nomn
NOUN,anim,femn,Patr sing,nomn


In [242]:
for item, row in data.iterrows():
    if row['gender'] == 'жен':
        print(row['head'])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Жива Базарова (Мункуева) Зинаида Хандажаповна 71 год, г. Улан-Удэ, Республика Бурятия
Re: Пропала Базарова (Мункуева) Зинаида Хандажаповна 71 год, г. Улан-Удэ, Республика Бурятия
Re: Пропала Базарова (Мункуева) Зинаида Хандажаповна 71 год, г. Улан-Удэ, Республика Бурятия
Жива Сяглова (Андреева) Татьяна Георгиевна 49 лет,  с. Жилино, Кабанский р-н
Re: Пропала Сяглова (Андреева) Татьяна Георгиевна 49 лет,  с. Жилино, Кабанский р-н
Погибла Банзаракцаева Аяна, 8 лет, г. Улан-Удэ
Re: Пропала Банзаракцаева Аяна, 8 лет, г. Улан-Удэ
Жива Митыпова (Гомбоева) Лора Ивановна 61 год, мкр.Го